In [1]:
import time
from functools import wraps
from typing import Any
from typing import Callable
from typing import Optional

### DEBUGGER


In [2]:
def debug(fn: Callable) -> Callable:
    @wraps(fn)
    def debugger(*args: Any, **kwargs: Any) -> Any:
        print(f"Args: {args}")
        print(f"Kwargs: {kwargs}")
        print(f"Function {fn.__name__} called")
        fn_result = fn(*args, **kwargs)
        print(f"Function {fn.__name__} returns: {fn_result}")
        return fn_result

    return debugger

In [3]:
@debug
def do_something(
    a: int,
    b: int,
    c: Optional[int] = None,
):
    return a + b if c else 0

In [4]:
@debug
def do_something2(
    a: int,
    b: int,
    c: Optional[int] = None,
):
    return a - b if c else 0

In [5]:
do_something(10, 20, c=1)

Args: (10, 20)
Kwargs: {'c': 1}
Function do_something called
Function do_something returns: 30


30

In [6]:
do_something2(44, 39, c=2)

Args: (44, 39)
Kwargs: {'c': 2}
Function do_something2 called
Function do_something2 returns: 5


5

#### TIMER


In [7]:
def timing(fn: Callable) -> Callable:
    @wraps(fn)
    def timer(*args: Any, **kwargs: Any) -> Any:
        print("Start timer!")
        start_time = time.perf_counter()
        fn_result = fn(*args, **kwargs)
        end_time = time.perf_counter()
        time_duration = end_time - start_time
        print(f"Function {fn.__name__} took: {time_duration} s")
        return fn_result

    return timer

In [8]:
@timing
def iterate(n: int) -> int:
    val = 0
    for i in range(n):
        val += i
    return val

In [9]:
iterate(1_000_000)

Start timer!
Function iterate took: 0.03932689999965078 s


499999500000

### Stacked Decorator


In [10]:
@debug
@timing
def my_function(name: str) -> None:
    print(f"Hello: {name}")

In [11]:
my_function("Jan")

Args: ('Jan',)
Kwargs: {}
Function my_function called
Start timer!
Hello: Jan
Function my_function took: 2.9000002541579306e-06 s
Function my_function returns: None


### Decorator Factory


In [12]:
def timing_extended(use_ns_timer: bool = False) -> Callable:
    if use_ns_timer:
        time_fn = time.perf_counter_ns
        time_scale = "ns"
    else:
        time_fn = time.perf_counter
        time_scale = "s"

    def timing(fn: Callable):  # Decorator
        @wraps(fn)
        def timer(*args: Any, **kwargs: Any):
            start_time = time_fn()
            fn_result = fn(*args, **kwargs)
            end_time = time_fn()
            time_duration = end_time - start_time
            print(f"Function {fn.__name__} took: {time_duration} {time_scale}")
            return fn_result

        return timer

    return timing

In [13]:
@debug
@timing_extended(use_ns_timer=True)
def my_function(name):
    print(f"Hello: {name}")

In [14]:
my_function("Jan")

Args: ('Jan',)
Kwargs: {}
Function my_function called
Hello: Jan
Function my_function took: 4100 ns
Function my_function returns: None
